## 네이버 지식인 크롤링

### 라이브러리 로드

In [1]:
from bs4 import BeautifulSoup  
import pandas as pd
import requests

### 지식인 URL 만들기

* 5페이지에 대한 url 생성


In [4]:
keyword = '파이썬'
page_num = 5

url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page_num}'

In [17]:
for num in range(page_num):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={num+1}'
    print(url)

https://kin.naver.com/search/list.naver?query=파이썬&page=1
https://kin.naver.com/search/list.naver?query=파이썬&page=2
https://kin.naver.com/search/list.naver?query=파이썬&page=3
https://kin.naver.com/search/list.naver?query=파이썬&page=4
https://kin.naver.com/search/list.naver?query=파이썬&page=5


### 검색 요청하고 결과 html 가져오기

In [18]:
html = requests.get(url)
html

<Response [200]>

### 검색 결과 파싱하기

In [20]:
soup = BeautifulSoup(html.content,'html.parser')
# soup

### 검색결과의 href 추출하기

In [28]:
products = soup.select("#s_content > div.section > ul > li > dl > dt > a")

In [ ]:
sub_url = products[0].get('href')
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=480820366&qb=7YyM7J207I2s&enc=utf8'

### Sub URL 요청하고 html 가져오기

In [198]:
html_sub = requests.get(sub_url)
soup_sub = BeautifulSoup(html_sub.content,'html.parser')

### Sub 제목 추출

In [137]:
sel = "#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.endTitleSection"
tit_text = soup_sub.select_one(sel).text
tit_text

'\n\n\n\n\n\n\n질문\n\n\t\t\t\t\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t맥북 파이썬 편집기 창 없음\n\t\t\t\t\t\n\t\t\t\t\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t'

In [141]:
tit_text = tit_text.replace('질문', '')
tit_text

'\n\n\n\n\n\n\n\n\n\t\t\t\t\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t맥북 파이썬 편집기 창 없음\n\t\t\t\t\t\n\t\t\t\t\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t'

In [142]:
# 정규화 함수
import re
re.sub('[^0-9a-zA-Zㄱ-ㅣ가-힣 ]','',tit_text)

'맥북 파이썬 편집기 창 없음'

In [143]:
re.sub('[^0-9a-zA-Zㄱ-ㅣ가-힣 ]','-',tit_text)

'-------------------------------------------------맥북 파이썬 편집기 창 없음--------------------------'

In [144]:
re.sub('[^0-9a-zA-Zㄱ-ㅣ가-힣 ]',' ',tit_text)

'                                                 맥북 파이썬 편집기 창 없음                          '

In [145]:
re.sub('[^0-9a-zA-Z ]','',tit_text)

'    '

In [146]:
re.sub('[0-9a-zA-Zㄱ-ㅣ가-힣 ]','',tit_text)

'\n\n\n\n\n\n\n\n\n\t\t\t\t\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\n\t\t\t\t\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t'

In [147]:
tit_text = re.sub('[^ㄱ-ㅣ가-힣 ]','',tit_text)
tit_text

'맥북 파이썬 편집기 창 없음'

### Sub 본문 추출

In [148]:
content_url = "#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.questionDetail"
content_text = soup_sub.select_one(content_url).text
content_text

'\n\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t이번에 맥북을 새로 사서 과제하려고 파이썬을 설치했는데요,\xa0 윈도우에서는 IDLE shell창에 편집기를 여는 창 (File Edit Shell Debug Options Window Help가 위에 떠있는 창) 이 있었는데맥북에 설치한 파이썬은 편집기를 여는 창이 없더라구요....과제를 편집기로 제출해야하는데, 이 경우에는 어떻게 해야할까요? 맥북에서는 편집기를 어떻게 열어야 하나요....?\n'

In [149]:
content_text = re.sub('[^0-9a-zA-Zㄱ-ㅣ가-힣 ]','',content_text)
content_text

'이번에 맥북을 새로 사서 과제하려고 파이썬을 설치했는데요 윈도우에서는 IDLE shell창에 편집기를 여는 창 File Edit Shell Debug Options Window Help가 위에 떠있는 창 이 있었는데맥북에 설치한 파이썬은 편집기를 여는 창이 없더라구요과제를 편집기로 제출해야하는데 이 경우에는 어떻게 해야할까요 맥북에서는 편집기를 어떻게 열어야 하나요'

### Sub 답변 추출

In [201]:
# 답변 
# answer_text_lst = soup_sub.select('#answer_2 > div._endContents.c-heading-answer__content')
answer_text_lst = soup_sub.select('#SE-ced734d9-d961-11ef-ac73-99bf45991ec1 > div > div > div')
answer_text_lst

[<div class="se-module se-module-text"> <!-- SE-TEXT { --><p class="se-text-paragraph se-text-paragraph-align-" id="SE-ced734db-d961-11ef-ac73-a1b6af0d8148" style=""><span class="se-fs- se-ff-" id="SE-0b803aeb-d962-11ef-ac73-a1984e5490f0" style="">맥북 처음이신가 보네요.</span></p><!-- } SE-TEXT --><!-- SE-TEXT { --><p class="se-text-paragraph se-text-paragraph-align-" id="SE-d43bd299-d961-11ef-ac73-71e0da5b1070" style=""><span class="se-fs- se-ff-" id="SE-0b803aec-d962-11ef-ac73-334711cd0ab4" style="">윈도우와 다르게 맥북은 툴바가 창 내에 있는게 아니라 가장 상단(애플로고 옆으로)에 있습니다.</span></p><!-- } SE-TEXT --> </div>]

In [202]:
len(answer_text_lst)

1

In [203]:
answer_text_lst[0].text

' 맥북 처음이신가 보네요.윈도우와 다르게 맥북은 툴바가 창 내에 있는게 아니라 가장 상단(애플로고 옆으로)에 있습니다. '

In [159]:
answer_text = ""
for i in answer_text_lst:
    answer = re.sub('[^a-zA-Zㄱ-ㅣ가-힣 ]','',i.text)
    answer_text = answer_text + " " + answer

In [160]:
answer_text.strip()

'맥북 처음이신가 보네요윈도우와 다르게 맥북은 툴바가 창 내에 있는게 아니라 가장 상단애플로고 옆으로에 있습니다'

------------

## 네이버 지식인 크롤링 자동화

In [40]:
# 1. 검색 결과 n페이지를 봐야하고 (검색어가 있어야 한다.)
#     2. n 하이퍼링크에 연결된 게시물로 이동
#     2-1. 제목을 추출
#     2-2. 본문을 추출(만약 본문이 없으면 '본문 없음')
#     2-3. 답변을 추출
#     2-3-1. 여러개의 답변일 경우 하나의 문자열로 만든다
#     3. 추출한 제목,본문,답변을 리스트에 추가

## 종합코드

In [2]:
import re
from bs4 import BeautifulSoup  
import pandas as pd
import requests


# 데이터 저장하기 위해 데이터프레임 생성
idx = 0
kin_df = pd.DataFrame(columns=['제목','본문','답변']) 

# URL 만들기
keyword = '파이썬'
page_num = 10

url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page_num}'

for num in range(page_num):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={num+1}'
    # print(url)
    html = requests.get(url)                                     # 검색 요청하고 HTML 가져오기
    soup = BeautifulSoup(html.content, 'html.parser')            # 검색 결과 파싱
    products = soup.select("#s_content > div.section > ul > li > dl > dt > a")

    for product in products:
        sub_url = product.get('href')                            # 서브 URL 추출
        # print(sub_url)
        html_sub = requests.get(sub_url)                         # 서브 URL 요청
        soup_sub = BeautifulSoup(html_sub.content,'html.parser') # HTML 가져오기
        
        # 서브 제목 추출
        sel = "#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.endTitleSection"
        tit_text = soup_sub.select_one(sel).text 
        tit_text = tit_text.replace('질문', '')
        tit_text = re.sub('[^ㄱ-ㅣ가-힣 ]','',tit_text)  # 문장 정규화
        
        # 서브 본문 추출
        content_url = "#content > div.endContentLeft._endContentLeft > div.contentArea._contentWrap > div.questionDetail"
        content_text = soup_sub.select_one(content_url).text
        content_text = re.sub('[^a-zA-Zㄱ-ㅣ가-힣 ]','', content_text) # 문장 정규화
        
        # 만약 본문이 없으면 '본문 없음'을 기록
        if content_text == '':
            content_text = '본문 없음'

        # 서브 답변 추출
        answer_text_lst = soup_sub.select('div.answerDetail._endContents._endContentsText')
        answer_text = re.sub('[^a-zA-Zㄱ-ㅣ가-힣 ]','', answer_text_lst[0].text).strip() # 문장 정규화

        # 데이터프레임 안에 데이터 기록
        idx += 1
        kin_df.loc[idx] = [tit_text,content_text,answer_text]

In [3]:
print(f"ex)검색결과 페이지: {url}")
print(f"ex)하이퍼링크에 연결된 게시물로 이동: {sub_url}")

ex)검색결과 페이지: https://kin.naver.com/search/list.naver?query=파이썬&page=10
ex)하이퍼링크에 연결된 게시물로 이동: https://kin.naver.com/qna/detail.naver?d1id=11&dirId=110409&docId=476492291&qb=7YyM7J207I2s&enc=utf8


In [4]:
kin_df.head(3)

,제목,본문,답변
1,파이썬강의로 배우기 괜찮을까요,이쪽 전문지식이 거의 없는 비전공자입니다그래서 아주 쉬운것부터 알려줄 파이썬강의가 ...,처음 배울때는 많이 어렵기 때문에 온라인으로 파이썬강의을 듣기보다는전문 교육기관에서...
2,파이썬,파이썬 open함수로 엑셀파일을 열수있나요,open 함수는 일반적인 텍스트 파일을 열기 위한 기본 함수입니다 엑셀 파일은 텍스...
3,언어 배우고 파이썬,C언어 기초를 직전 학기에 들었는데 복전 조건 강의가 파이썬 기초인데 C언어랑 파이...,C언어와 파이썬은 기본적인 프로그래밍 개념변수 연산자 조건문 반복문 함수 등은 유사...


In [5]:
kin_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 1 to 100
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      100 non-null    object
 1   본문      100 non-null    object
 2   답변      100 non-null    object
dtypes: object(3)
memory usage: 3.1+ KB


## 데이터프레임 저장

In [7]:
kin_df.to_csv('./data/JK_파이썬_지식인크롤링.csv')